# Prohledávání hyperparametrů pro model MobileNetV2 nad datasetem CIFAR100

Tento notebook slouží k nalezení optimálních hyperparametrů nad datasetem CIFAR100 pro model MobileNetV2. Hyperparametry jsou hledány pro všechny varianty modelu (náhodně inicializovaný, předtrénovaný (doučení klasifikační hlavy) a předtrénovaný (doučení celého modelu)). Pro každou z variant jsou hledány hyperparametry pro normální trénink a trénink s destilací. 

K prohledávání je využito knihovny Optuna s algoritmem Hyperband. Nejlepší konfigurace je volena na základě F1-skóre, zkoušeno je 150 kombinací hyperparametrů pro každou variantu modelu. 

## Import knihoven a základní nastavení

In [16]:
from transformers import Trainer
import optuna
import torch
import math
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [17]:
dataset_part = base.get_dataset_part()

Resetování náhodného seedu pro replikovatelnost výsledků.

In [18]:
base.reset_seed()

Ověření dostupnosti GPU.

In [19]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Načtení datasetu a aplikace základních transformací.

In [20]:
DATASET = "cifar100"

In [21]:
transform = base.base_transforms()

train = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TRAIN, transform=transform, device="cpu")
eval = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.EVAL, transform=transform, device="cpu")
test = base.CustomCIFAR100L(root=f"{os.path.expanduser('~')}/data/100-logits", dataset_part=dataset_part.TEST, transform=transform, device="cpu")

Základní konfigurace tréninku během prohledávání. Optuna nepracuje s epochami, ale s kroky. Níže je prováděn přepočet. 

Minimální délka tréninku jsou tři epochy, maximální deset epoch. Maximální počet kroků pro warm up je nastaven na 10 % první epochy.

In [23]:
num_epochs = 10
batch_size = 128

In [24]:
data_length = len(train)
min_r = math.ceil(data_length/batch_size)*3
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

## Prohledávání s normálním tréninkem náhodně inicializovaného modelu
Definice hledaných hyperparametrů a jejich rozmezí.

In [25]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [26]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [27]:
base.reset_seed()

Konfigurace jednotlivých tréninků.

In [28]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random_hp-search", logging_dir=f"~/logs/{DATASET}/random_hp-search", epochs=num_epochs, batch_size=batch_size)

Konfigurace trenéra pro jednotlivé tréninky. 

In [29]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_random_init_mobilenet(100)
)
  

Nastavení prohledávání.

In [ ]:
best_base_random = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-random",
    n_trials=150
)

[I 2025-03-25 07:19:45,527] A new study created in memory with name: Base-random


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,4.219200,3.577190,0.136600,0.120910,0.136600,0.099373
2,3.507400,3.112422,0.228400,0.225727,0.228400,0.204657
3,3.026800,2.673009,0.309400,0.299968,0.309400,0.282376


[I 2025-03-25 07:23:35,183] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Trial 90 with params: {'learning_rate': 0.0011372885658903864, 'weight_decay': 0.007, 'warmup_steps': 13}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,4.235200,3.879623,0.076200,0.059154,0.076200,0.049869
2,3.812500,3.554902,0.134400,0.122592,0.134400,0.107791
3,3.455000,3.119169,0.213900,0.208830,0.213900,0.183545


[I 2025-03-25 20:46:15,281] Trial 90 pruned. 


Trial 91 with params: {'learning_rate': 0.00027658201251049946, 'weight_decay': 0.008, 'warmup_steps': 5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,4.211500,3.638175,0.124600,0.114905,0.124600,0.089547
2,3.492400,3.093611,0.228400,0.227999,0.228400,0.202603
3,3.009600,2.637584,0.316700,0.307555,0.316700,0.291020
4,2.616400,2.485863,0.345700,0.367451,0.345700,0.329566
5,2.314300,2.196630,0.413200,0.414250,0.413200,0.399587
6,2.033500,2.097523,0.439300,0.442810,0.439300,0.430081
7,1.791000,2.033218,0.454400,0.459722,0.454400,0.446411
8,1.549400,2.033970,0.454200,0.465649,0.454200,0.446274
9,1.353300,2.068846,0.451000,0.455025,0.451000,0.439894


In [ ]:
print(best_base_random)

In [18]:
base.reset_seed()

## Prohledávání s destilací náhodně inicializovaného modelu

In [19]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random-KD_hp-search", logging_dir=f"~/logs/{DATASET}/random-KD_hp-search",  remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [20]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [21]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [22]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_random_init_mobilenet(100)
)
  

Nastavení prohledávání.

In [ ]:
best_distill_random = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-25 21:17:10,361] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.946300,3.468593,0.130000,0.099117,0.130000,0.088588
2,3.380600,3.036226,0.231600,0.226119,0.231600,0.198101
3,2.970700,2.649721,0.319500,0.311019,0.319500,0.284502


[I 2025-03-25 21:20:59,280] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.940400,3.731105,0.063400,0.040018,0.063400,0.030336
2,3.625900,3.464541,0.131200,0.108167,0.131200,0.095815
3,3.362000,3.166537,0.193600,0.180103,0.193600,0.155711


[I 2025-03-26 19:54:54,303] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.00036175490456492393, 'weight_decay': 0.002, 'warmup_steps': 22, 'lambda_param': 0.8, 'temperature': 4.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.910700,3.435325,0.128200,0.097487,0.128200,0.084195
2,3.302500,2.953835,0.256700,0.262607,0.256700,0.229138
3,2.894400,2.580575,0.330900,0.323724,0.330900,0.299945
4,2.568900,2.482928,0.355500,0.391606,0.355500,0.333282
5,2.306700,2.183867,0.436700,0.441992,0.436700,0.420007
6,2.068200,2.063210,0.467000,0.474565,0.467000,0.459876
7,1.861400,1.984611,0.482100,0.490378,0.482100,0.474019
8,1.665900,1.966948,0.488300,0.493588,0.488300,0.478705
9,1.499900,2.007833,0.480400,0.487065,0.480400,0.471535
10,1.373000,1.897270,0.505500,0.515627,0.505500,0.503846


[I 2025-03-26 20:07:22,408] Trial 149 finished with value: 0.5038455837632714 and parameters: {'learning_rate': 0.00036175490456492393, 'weight_decay': 0.002, 'warmup_steps': 22, 'lambda_param': 0.8, 'temperature': 4.5}. Best is trial 122 with value: 0.5182847790196461.


In [ ]:
print(best_distill_random)

BestRun(run_id='122', objective=0.5182847790196461, hyperparameters={'learning_rate': 0.00049497207425666, 'weight_decay': 0.005, 'warmup_steps': 0, 'lambda_param': 0.2, 'temperature': 3.0}, run_summary=None)


In [ ]:
base.reset_seed()

## Prohledávání s normálním tréninkem s doučením klasifikační hlavy předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-head_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-head_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [ ]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.freeze_model(base.get_mobilenet(100))
)
  

Nastavení prohledávání.

In [ ]:
best_base_head = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-26 20:07:23,117] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.520200,2.770046,0.422500,0.468604,0.422500,0.406531
2,2.352700,2.199290,0.477800,0.497693,0.477800,0.473857
3,2.006600,2.018175,0.504900,0.507880,0.504900,0.495592


[I 2025-03-26 20:09:48,407] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.494200,1.966831,0.492200,0.542861,0.492200,0.482354
2,1.696600,1.814022,0.518000,0.547152,0.518000,0.517962
3,1.530800,1.748888,0.533700,0.550168,0.533700,0.527386
4,1.436100,1.810481,0.521000,0.550129,0.521000,0.518430
5,1.373000,1.729006,0.537900,0.556396,0.537900,0.533450
6,1.317500,1.672747,0.548900,0.554458,0.548900,0.544677


[I 2025-03-27 10:34:19,828] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0013581673151824224, 'weight_decay': 0.003, 'warmup_steps': 11}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.468900,1.956570,0.492300,0.545856,0.492300,0.482804
2,1.686400,1.812019,0.518000,0.548718,0.518000,0.518284
3,1.522300,1.750192,0.535000,0.552113,0.535000,0.528476
4,1.427900,1.814465,0.520300,0.550231,0.520300,0.517937
5,1.364500,1.732801,0.537100,0.556802,0.537100,0.533058
6,1.308200,1.673868,0.547600,0.553784,0.547600,0.543542


[I 2025-03-27 10:39:05,651] Trial 149 pruned. 


In [ ]:
print(best_base_head)

BestRun(run_id='53', objective=0.5431165419813282, hyperparameters={'learning_rate': 0.0007167369272358462, 'weight_decay': 0.001, 'warmup_steps': 9}, run_summary=None)


In [ ]:
base.reset_seed()

## Prohledávání s destilací s doučením klasifikační hlavy předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-head-KD_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-head-KD_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [ ]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.freeze_model(base.get_mobilenet(100))
)

Nastavení prohledávání.

In [ ]:
best_distill_head = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-27 10:39:06,404] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.360400,2.799643,0.421600,0.485530,0.421600,0.405836
2,2.517800,2.410802,0.484000,0.500740,0.484000,0.477515
3,2.288400,2.284865,0.506100,0.510731,0.506100,0.493501


[I 2025-03-27 10:41:33,114] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.589800,2.220660,0.502800,0.545143,0.502800,0.496953
2,2.106100,2.161384,0.515900,0.551013,0.515900,0.515369
3,2.027700,2.113755,0.533600,0.548658,0.533600,0.524268
4,1.979700,2.148149,0.521300,0.545588,0.521300,0.515823
5,1.950400,2.095211,0.531700,0.550096,0.531700,0.525890
6,1.915200,2.042478,0.548500,0.558222,0.548500,0.543707
7,1.890500,2.026643,0.549400,0.553854,0.549400,0.544785
8,1.866400,2.068042,0.540100,0.545653,0.540100,0.533290
9,1.841500,2.069819,0.542400,0.551241,0.542400,0.537369
10,1.825000,2.054297,0.541600,0.548408,0.541600,0.538247


[I 2025-03-28 00:47:32,252] Trial 148 finished with value: 0.5382470949316869 and parameters: {'learning_rate': 0.0019413182155920852, 'weight_decay': 0.007, 'warmup_steps': 25, 'lambda_param': 0.0, 'temperature': 5.5}. Best is trial 100 with value: 0.5388203850987133.


Trial 149 with params: {'learning_rate': 0.0011364238732391222, 'weight_decay': 0.005, 'warmup_steps': 11, 'lambda_param': 0.5, 'temperature': 4.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.674500,2.268925,0.495700,0.537291,0.495700,0.483894
2,2.119800,2.159220,0.518900,0.540617,0.518900,0.516849
3,2.024400,2.102021,0.533000,0.542633,0.533000,0.523896
4,1.972800,2.142345,0.522700,0.546116,0.522700,0.517547
5,1.942300,2.080982,0.536300,0.548682,0.536300,0.529658
6,1.910300,2.036973,0.548900,0.552201,0.548900,0.543148
7,1.890000,2.023242,0.551900,0.553307,0.551900,0.546608
8,1.871700,2.067098,0.540800,0.544753,0.540800,0.533915
9,1.853700,2.067748,0.542800,0.550425,0.542800,0.537720
10,1.843900,2.059108,0.540500,0.547194,0.540500,0.536829


[I 2025-03-28 00:55:31,763] Trial 149 finished with value: 0.5368289292477528 and parameters: {'learning_rate': 0.0011364238732391222, 'weight_decay': 0.005, 'warmup_steps': 11, 'lambda_param': 0.5, 'temperature': 4.0}. Best is trial 100 with value: 0.5388203850987133.


In [ ]:
print(best_distill_head)

BestRun(run_id='100', objective=0.5388203850987133, hyperparameters={'learning_rate': 0.0018015420304320652, 'weight_decay': 0.008, 'warmup_steps': 31, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}, run_summary=None)


In [ ]:
base.reset_seed()

## Prohledávání s normálním tréninkem s doučením předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [ ]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_mobilenet(100)
)
  

Nastavení prohledávání.

In [ ]:
best_base_pretrained = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-28 00:55:32,395] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.853300,1.021426,0.699700,0.717809,0.699700,0.697692
2,0.722500,0.966209,0.717400,0.743706,0.717400,0.714577
3,0.415300,0.898850,0.741500,0.758191,0.741500,0.739484


[I 2025-03-28 00:59:16,343] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

[I 2025-03-28 14:45:36,089] Trial 96 finished with value: 0.7791320903992918 and parameters: {'learning_rate': 0.00043054586762287123, 'weight_decay': 0.009000000000000001, 'warmup_steps': 10}. Best is trial 67 with value: 0.7793985767219671.


Trial 97 with params: {'learning_rate': 0.000766818047088654, 'weight_decay': 0.007, 'warmup_steps': 14}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.660300,1.254248,0.643200,0.683538,0.643200,0.637446
2,0.868500,1.057422,0.692700,0.720458,0.692700,0.691763


: 

: 

In [ ]:
print(best_base_pretrained)

In [ ]:
base.reset_seed()

## Prohledávání s destilací s doučením předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-KD_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-KD_hp-search",  remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [ ]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_mobilenet(100)
)

Nastavení prohledávání.

In [ ]:
best_distil_pretrained = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-28 19:35:28,117] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.004346495755569775, 'weight_decay': 0.008, 'warmup_steps': 31, 'lambda_param': 0.9, 'temperature': 5.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.464500,2.034819,0.473500,0.516155,0.473500,0.459134
2,1.744200,1.678343,0.565500,0.609680,0.565500,0.563037
3,1.405200,1.454783,0.620600,0.643512,0.620600,0.612996


[I 2025-03-28 19:39:15,438] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.003489140632563016, 'weight_decay': 0.0, 'warmup_steps': 6, 'lambda_param': 0.0, 'temperature': 3.5}


...

[I 2025-03-29 06:04:52,609] Trial 72 finished with value: 0.7878383506232299 and parameters: {'learning_rate': 0.0005309673044490899, 'weight_decay': 0.008, 'warmup_steps': 31, 'lambda_param': 0.1, 'temperature': 4.0}. Best is trial 35 with value: 0.7904627692423465.


Trial 73 with params: {'learning_rate': 0.00015972356535382792, 'weight_decay': 0.01, 'warmup_steps': 2, 'lambda_param': 0.1, 'temperature': 6.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.105700,1.259991,0.679500,0.691969,0.679500,0.674568
2,0.991600,1.080157,0.720500,0.738830,0.720500,0.718935
3,0.688900,0.968549,0.741200,0.753398,0.741200,0.739046
4,0.512100,0.982014,0.735300,0.755110,0.735300,0.736424
5,0.398600,0.909502,0.756800,0.767659,0.756800,0.754320


[W 2025-03-29 06:12:23,882] Trial 73 failed with parameters: {'learning_rate': 0.00015972356535382792, 'weight_decay': 0.01, 'warmup_steps': 2, 'lambda_param': 0.1, 'temperature': 6.0} because of the following error: FileNotFoundError(2, 'No such file or directory').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 250, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2241, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2639, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, grad_norm, model, trial, epoch, ignore_keys_for_eval, start_time)
  File "/usr/local/lib/python3.1

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/.cache/huggingface/metrics/accuracy/default/default_experiment-1-0.arrow'

In [ ]:
print(best_distil_pretrained)

In [ ]:
print("Best random init training score: ", best_base_random)
print("Best random init distilation trianing score: ", best_distill_random)
print("Best pretrained (head only) training score: ", best_base_head)
print("Best pretrained distilation (head only) training score: ",best_distill_head)
print("Best pretrained training score: ", best_base_pretrained)
print("Best pretrained distilation training score: ", best_distil_pretrained)